In [ ]:
# instructions:
#figure out your parameters in the single version, parameters go in the variables minmass, maxsize and noise size
#place files in the same folder and the script will find them.
#the first block does all the tracking and it takes a while
#second graphs so you dont have to wait to track again if you want to change the plot.

In [ ]:

%pylab inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pdz
from pandas import DataFrame, Series  # for convenience
import IPython
import pims
import trackpy as tp
import av
import os
from os import listdir
from os.path import isfile, join, abspath, basename
import scipy as sp

mpl.rc('figure',  figsize=(10, 5))
mpl.rc('image', cmap='gray')

tp.linking.Linker.MAX_SUB_NET_SIZE = 50 #set max subnet size for tracking. comp time scales with n! will cause errors if not large enough

##pipelines to convert and crop images##
@pims.pipeline
def gray(image):
    return image[:, :, 2]  # Take just the green channel

@pims.pipeline
def crop(image,xmin,xmax,ymin,ymax):
    return image[xmin:xmax,ymin:ymax]



#tuning variables, pick these with single particle tracking notebook#
minmass = 18 #scales with the size
maxsize = 15
noise_size = 3
inverted = True
aproxsize = 5 #must be odd
mpp = .2037489813*60/40 #microns per pixel
fps = 1
######################################################################


try:
    path = sys.path[0]
    allfiles = [f for f in listdir(path) if isfile(join(path, f))]
    files = [ fi for fi in allfiles if fi.endswith(".tif") ]
    print("files opened succesfully")
    print("files: ",files)
except:
    print("oh no error opening files")



In [ ]:
t1 =[] #tracks storage
filenames = []

for fl in files:
    print(abspath(fl))
    filenames.append(basename(abspath(fl)))
    vid = pims.open(basename(abspath(fl))) 
    f = tp.batch(vid, aproxsize, invert=inverted, minmass=minmass, maxsize=maxsize, noise_size=noise_size)
    t = tp.link(f, 30, memory=3) #link them together
    t1 = t
    #t1.append(tp.filter_stubs(t, 5)) #gets rid of small trajectories

In [ ]:
#makes plots
os.chdir(path)
plotspath = os.path.join(path,"Plots")
if os.path.exists(plotspath):
    os.chdir(plotspath)
else:
    os.mkdir(plotspath)
    os.chdir(plotspath)
for i, tra in enumerate(t1):
    #drift positions
    d = tp.compute_drift(tra, smoothing = 5) #find drift
    d.plot()
    plt.xlabel("Frame Number")
    plt.ylabel("Position (px)")
    plt.title("Ensamble Drift Positions "+ filenames[i])
    plt.savefig("Ensamble Drift Position "+ filenames[i]+".png", format='png')
    plt.show()

    #single particle msd graph
    im = tp.imsd(tra, mpp, fps)
    fig, ax = plt.subplots()
    ax.plot(im.index, im, 'k-', alpha=0.1)  # black lines, semitransparent
    ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
       xlabel='lag time $t$')
    ax.set_xscale('log')
    ax.set_yscale('log')
    plt.title("Single Particle MSD "+ filenames[i])
    plt.savefig("Single Particle MSD "+ filenames[i]+".png", format='png')
    plt.show()

    #ensamble msd graph
    em = tp.emsd(tra, mpp, fps, detail=False)
    fig, ax = plt.subplots()
    ax.plot(em.index, em, 'o')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
        xlabel='lag time $t$')
    plt.title("Ensamble MSD "+ filenames[i])
    plt.savefig("Ensamble MSD "+ filenames[i]+".png", format='png')
    plt.show()

    #ensamble msd errors

    detailed_emsd = tp.emsd(tra, mpp, fps, detail=True)
    fig, ax = plt.subplots()
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]', xlabel='lag time $t$')
    plt.errorbar(detailed_emsd["lagt"].to_numpy(), 
        detailed_emsd["msd"].to_numpy(), 
        #yerr = detailed_emsd["<y^2>"].to_numpy(),
        yerr = np.sqrt(detailed_emsd["<y^2>"].to_numpy() + detailed_emsd["<x^2>"].to_numpy()), 
        )
    plt.title("Ensamble MSD Error"+ filenames[i])
    plt.savefig("Ensamble MSD Error"+ filenames[i]+".png", format='png')
    plt.show()

os.chdir("..")

In [ ]:
#plots one at a time to preserve memory
plotspath = os.path.join(path,"Plots")
if os.path.exists(plotspath):
    os.chdir(plotspath)
else:
    os.mkdir(plotspath)
    os.chdir(plotspath)

for fl in files:
    try: #sometimes theres errors for divide by zero or too many traces, this makes it trudge on through
        os.chdir(path)
        vid = pims.open(basename(abspath(fl))) 
        f = tp.batch(vid, aproxsize, invert=True, minmass=minmass, maxsize=maxsize, noise_size=noise_size)
        tra = tp.link(f, 18, memory=2) #link them together
        #t = tp.filter_stubs(t, 5) #gets rid of small trajectories

        os.chdir(plotspath)
        
        d = tp.compute_drift(tra, smoothing = 5) #find drift
        d.plot()
        plt.xlabel("Frame Number")
        plt.ylabel("Position (px)")
        plt.title("Ensamble Drift Positions "+ basename(abspath(fl)))
        plt.savefig("Ensamble Drift Position "+ basename(abspath(fl)) +".png", format='png')
        plt.show()

        #single particle msd graph
        im = tp.imsd(tra, mpp, fps)
        fig, ax = plt.subplots()
        ax.plot(im.index, im, 'k-', alpha=0.1)  # black lines, semitransparent
        ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
        xlabel='lag time $t$')
        ax.set_xscale('log')
        ax.set_yscale('log')
        plt.title("Single Particle MSD "+ basename(abspath(fl)))
        plt.savefig("Single Particle MSD "+ basename(abspath(fl)) +".png", format='png')
        plt.show()

        #ensamble msd graph
        em = tp.emsd(tra, mpp, fps, detail=False)
        fig, ax = plt.subplots()
        ax.plot(em.index, em, 'o')
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
            xlabel='lag time $t$')
        plt.title("Ensamble MSD "+ basename(abspath(fl)))
        plt.savefig("Ensamble MSD "+ basename(abspath(fl)) +".png", format='png')
        plt.show()

        #ensamble msd errors and fitting graph on top

        detailed_emsd = tp.emsd(tra, mpp, fps, detail=True)
        fig, ax = plt.subplots()
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]', xlabel='lag time $t$')
        plt.errorbar(detailed_emsd["lagt"].to_numpy(), 
            detailed_emsd["msd"].to_numpy(), 
            yerr = detailed_emsd["<y^2>"].to_numpy(),
            #yerr = detailed_emsd["<y^2>"].to_numpy() + detailed_emsd["<x^2>"].to_numpy(), 
            )

        #fitting
        lagt = detailed_emsd["lagt"].to_numpy()
        fit = tp.utils.fit_powerlaw(tp.emsd(tra, mpp, fps, detail=False),plot=False)
        n = fit.at["msd","n"]
        A = fit.at["msd","A"]
        fitline = A*lagt**n
        ax.plot(lagt,fitline, label = "D (2D): "+ str(np.round(A/4, decimals=4)))
        ax.legend()
        
        plt.title("Ensamble MSD Error"+ basename(abspath(fl)))
        plt.savefig("Ensamble MSD Error"+ basename(abspath(fl)) +".png", format='png')
        plt.show()
        
        #manual weighted fitting

        def fit(lagt, A, n):
            return A*lagt**n

        lagt = detailed_emsd["lagt"].to_numpy()
        msd = detailed_emsd["msd"].to_numpy()

        [A, n], pcov = sp.optimize.curve_fit(fit, lagt, msd, sigma=detailed_emsd["<y^2>"].to_numpy(), p0=None)
        err = np.sqrt(np.diag(pcov))

        def parafit(lagt, a ,b ,c):
            return np.exp( c + b*np.log(lagt) + a*np.log(lagt)**2 )


        [a, b, c], pcov = sp.optimize.curve_fit(parafit, lagt, msd, sigma=detailed_emsd["<y^2>"].to_numpy(), p0=None)
        paraerror = np.sqrt(np.diag(pcov))

        fig, ax = plt.subplots()
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]', xlabel='lag time $t$')
        plt.errorbar(detailed_emsd["lagt"].to_numpy(), 
            detailed_emsd["msd"].to_numpy(), 
            yerr = detailed_emsd["<y^2>"].to_numpy(),
            #yerr = detailed_emsd["<y^2>"].to_numpy() + detailed_emsd["<x^2>"].to_numpy(), 
            )
        fitline = fit(lagt, A, n)
        parafit = parafit(lagt,a,b,c)
        ax.plot(lagt,fitline, label = "D (2D): "+ str(np.round(A/4, decimals=4)) + " Error: " + str(np.round(err ,decimals=2)))
        ax.plot(lagt,parafit, label = "Parabolic" + str( np.round([a,b,c],decimals=2)) + " Error: " + str(np.round(paraerror ,decimals=2)))
        ax.legend()
        
        plt.title("Ensamble MSD Error"+ basename(abspath(fl)))
        plt.savefig("Ensamble MSD Error"+ basename(abspath(fl)) +".png", format='png')
        plt.show()
        
    except:
        print(basename(abspath(fl)), " failed")
    #break
os.chdir("..")